In [32]:
# Import the required libraries to load and execute the code
from os import makedirs
from os.path import join, exists, expanduser

from PIL import Image

import pandas as pd

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PyAedatTools.ImportAedat import ImportAedat
from PyAedatTools.ImportAedatHeaders import ImportAedatHeaders
from PyAedatTools.ImportAedatDataVersion1or2 import ImportAedatDataVersion1or2

In [33]:
def ConvertTDeventsToAccumulatedFrame(state, td_events, width=346, height=260, tau=1e6):
    """ Converts the specified TD data to a frame by accumulating events """
    if state is None:
        state = {}
        state['last_spike_time'] = np.zeros([width, height])
        state['last_spike_polarity'] = np.zeros([width, height])
    td_image = np.zeros([width, height])
    
    last_t = 0
    for row in td_events:
        [x, y, p, t] = row
    
        state['last_spike_time'][x, y] = t
        state['last_spike_polarity'][x, y] = 1 if p == True else -1
        last_t = t 
        
        td_image[x,y] = p
    
    surface_image = state['last_spike_polarity']*np.exp((state['last_spike_time'] - last_t) / tau)
    
    #return [state, td_image.transpose()]
    return [state, surface_image.transpose()]

In [34]:
# Configure the input and output folders
aedat_file = "Tobi1"
input_file_path = join(expanduser("~"), "TelGanData", "{}.aedat".format(aedat_file))
output_folder = join(expanduser("~"), "TelGanData", aedat_file)

generator_image_folder = join(output_folder, 'generator')
target_image_folder = join(output_folder, 'targets')
combined_image_folder = join(output_folder, 'combined')
if not exists(generator_image_folder):
    makedirs(generator_image_folder)
if not exists(target_image_folder):
    makedirs(target_image_folder)
if not exists(combined_image_folder):
    makedirs(combined_image_folder)

In [35]:
# Configure the reading parameters
aedat = {}
aedat['importParams'] = {}
aedat['importParams']['filePath'] = input_file_path

# Configure which parts of the file to read
aedat['importParams']['startEvent'] = int(1e6);
aedat['importParams']['endEvent'] = int(10e6); 
#aedat['importParams']['startTime'] = 48;
#aedat['importParams']['endTime'] = 49;

aedat['importParams']['dataTypes'] = {'polarity', 'special', 'frame'};

In [36]:
# Invoke the function
aedat = ImportAedat(aedat)
print('Read {} seconds of data'.format((aedat['info']['lastTimeStamp'] - aedat['info']['firstTimeStamp']) / 1e6))
print('Read {} events.'.format(aedat['info']['lastTimeStamp'] - aedat['info']['firstTimeStamp']))
print('Done!')

Reading events ...
Building logical indices by type ...
Processing polarity events ...
Processing frames ...
Processing frame  10  of  99
Processing frame  20  of  99
Processing frame  30  of  99
Processing frame  40  of  99
Processing frame  50  of  99
Processing frame  60  of  99
Processing frame  70  of  99
Processing frame  80  of  99
Processing frame  90  of  99
Performing subtraction on frame  10  of  99
Performing subtraction on frame  20  of  99
Performing subtraction on frame  30  of  99
Performing subtraction on frame  40  of  99
Performing subtraction on frame  50  of  99
Performing subtraction on frame  60  of  99
Performing subtraction on frame  70  of  99
Performing subtraction on frame  80  of  99
Performing subtraction on frame  90  of  99
Read 4.884972 seconds of data
Read 4884972 events.
Done!


In [37]:
num_frames = aedat['data']['frame']['numEvents']
num_events = aedat['info']['lastTimeStamp'] - aedat['info']['firstTimeStamp']
assert(num_frames > 0)
assert(num_frames == len(aedat['data']['frame']['samples']))

last_frame_start = aedat['data']['frame']['timeStampStart'][0]
last_frame_end = aedat['data']['frame']['timeStampEnd'][0]
last_td_timestamp_index = 0
last_td_timestamp = aedat['data']['polarity']['timeStamp'][last_td_timestamp_index]

update_interval = round(max(num_frames / 100, 1))

event_data = []

event_id = 0

# frame = [0,1], event_id, prev_frame_id, x, y, value, timestamp

# Loop through the frames and save them accordingly
for frame_counter in range(1, num_frames):
    
    # Find out when the frame started and ended
    frame_start = aedat['data']['frame']['timeStampStart'][frame_counter]
    frame_end = aedat['data']['frame']['timeStampEnd'][frame_counter]
    
    # Figure out the range of TD times that we are interested in
    assert(last_frame_end <= frame_start)
    td_range = [last_frame_end, frame_start]
    last_frame_start = aedat['data']['frame']['timeStampStart'][frame_counter]
    last_frame_end = aedat['data']['frame']['timeStampEnd'][frame_counter]    
    
    # Extract the TD region for this frame
    assert(last_td_timestamp <= td_range[0]) # Ensure that we're seeking to a time in the future
    # Find the starting event
    while (last_td_timestamp < td_range[0] and last_td_timestamp_index < num_events):
        last_td_timestamp_index = last_td_timestamp_index + 1
        last_td_timestamp = aedat['data']['polarity']['timeStamp'][last_td_timestamp_index]
    # Now find the last event and store all intermediate events in the array
    
    td_events = []
    frame_now = 1
    while (last_td_timestamp < td_range[1] and last_td_timestamp_index < num_events):
        td_events.append(np.array([frame_now,
                          event_id,
                          frame_counter, 
                          aedat['data']['polarity']['x'][last_td_timestamp_index],
                          aedat['data']['polarity']['y'][last_td_timestamp_index],
                          aedat['data']['polarity']['polarity'][last_td_timestamp_index],
                          aedat['data']['polarity']['timeStamp'][last_td_timestamp_index]])
        )
        event_id += 1
        frame_now = 0
        last_td_timestamp_index = last_td_timestamp_index + 1
        last_td_timestamp = aedat['data']['polarity']['timeStamp'][last_td_timestamp_index]
    # Convert the events to a numpy array
    #td_events = np.array(td_events)
    
    event_data += td_events
    
    print('Frame: {} Read {} events from time {} to time {}'.format(frame_counter, len(td_events), td_range[0], td_range[1]))
    state = None
    
df = pd.DataFrame(np.array(event_data), columns=["frame_now", "event_id", "prev_frame", "x", "y", "polarity", "timestamp"])
df.to_csv(join(output_folder, "{}.csv".format(aedat_file)), index=False)




Frame: 1 Read 1907 events from time 247244327 to time 247335076
Frame: 2 Read 2883 events from time 247344048 to time 247434797
Frame: 3 Read 2221 events from time 247443769 to time 247534517
Frame: 4 Read 4587 events from time 247543489 to time 247634238
Frame: 5 Read 5754 events from time 247643210 to time 247733959
Frame: 6 Read 5267 events from time 247742931 to time 247833680
Frame: 7 Read 4067 events from time 247842657 to time 247933401
Frame: 8 Read 4641 events from time 247942372 to time 248033121
Frame: 9 Read 3319 events from time 248042093 to time 248132842
Frame: 10 Read 2753 events from time 248141820 to time 248232563
Frame: 11 Read 1896 events from time 248241534 to time 248332284
Frame: 12 Read 1434 events from time 248341255 to time 248432005
Frame: 13 Read 2026 events from time 248440976 to time 248531725
Frame: 14 Read 2255 events from time 248540697 to time 248631446
Frame: 15 Read 1590 events from time 248640417 to time 248731167
Frame: 16 Read 1367 events from ti